In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import keras as kr


In [11]:
biden_v_trump = pd.read_csv('../election_resources/historical_elections.csv')
biden_v_trump.head()

,samplesize,party,results,favorability
0,1204.0,d,1.0,-22.0006
1,1553.0,d,1.0,-2.0001
2,838.0,d,1.0,-17.6672
3,1198.0,d,1.0,-20.3339
4,1859.0,d,1.0,-6.6670


In [12]:
biden_v_trump.head()

,samplesize,party,results,favorability
0,1204.0,d,1.0,-22.0006
1,1553.0,d,1.0,-2.0001
2,838.0,d,1.0,-17.6672
3,1198.0,d,1.0,-20.3339
4,1859.0,d,1.0,-6.6670


In [13]:
biden_v_trump.columns

Index(['samplesize', 'party', 'results', 'favorability'], dtype='object')

In [14]:
biden_v_trump.nunique()

samplesize       795
party              2
results            2
favorability    6335
dtype: int64

In [15]:
biden_v_trump['samplesize'].value_counts()

1000.0                                              305
1500.0                                              144
2001.0                                               55
900.0                                                31
1183.0                                               26
                                                   ... 
846.0                                                 1
1978.0                                                1
1289.0                                                1
760.0                                                 1
>>>>>>> c6fdee4765df870028d1ec168a99f34f4635955b      1
Name: samplesize, Length: 795, dtype: int64

In [16]:
sample_size_to_replace = list(biden_v_trump['samplesize'].value_counts()[biden_v_trump['samplesize'].value_counts() < 306].index)
for i in sample_size_to_replace:
    biden_v_trump['samplesize'] = biden_v_trump['samplesize'].replace(i, 'samplesize')
biden_v_trump['samplesize'].value_counts()

samplesize    6443
Name: samplesize, dtype: int64

In [17]:
biden_v_trump.shape

(6443, 4)

In [18]:
biden_v_trump = pd.get_dummies(biden_v_trump, dtype=float)
biden_v_trump.head()

,results,favorability,samplesize_samplesize,party_d,party_r
0,1.0,-22.0006,1.0,1.0,0.0
1,1.0,-2.0001,1.0,1.0,0.0
2,1.0,-17.6672,1.0,1.0,0.0
3,1.0,-20.3339,1.0,1.0,0.0
4,1.0,-6.6670,1.0,1.0,0.0


In [19]:
biden_v_trump.fillna(0, inplace=True)

In [20]:
biden_v_trump.shape

(6443, 5)

In [21]:
y = biden_v_trump['results'].values
X = biden_v_trump.drop(['results'], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

In [22]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
X_scaled.shape

(4832, 4)

In [24]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation', ['tanh', 'sigmoid', 'leaky_relu', 'elu', 'selu', 'PReLU'])
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
                                        min_value=1,
                                        max_value=30,
                                        step=2), activation=activation, input_dim=len(X_scaled[0])))
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                        min_value=1,
                                        max_value=30,
                                        step=2), activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    loss = hp.Choice('loss', ['binary_crossentropy', 'mse'])
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd', 'adagrad', 'adadelta', 'adamax', 'nadam'])
    nn_model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    
    return nn_model

In [25]:
tuner = kt.Hyperband(create_model, objective='val_accuracy', max_epochs=20, hyperband_iterations=10, directory='my_dir2', project_name='intro_to_kt')

In [26]:
tuner.search(X_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 300 Complete [00h 00m 17s]
val_accuracy: 0.9689633846282959

Best val_accuracy So Far: 0.9782743453979492
Total elapsed time: 00h 47m 50s
INFO:tensorflow:Oracle triggered exit


In [27]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'selu',
 'first_units': 25,
 'num_layers': 7,
 'units_0': 21,
 'loss': 'binary_crossentropy',
 'optimizer': 'rmsprop',
 'units_1': 17,
 'units_2': 23,
 'units_3': 23,
 'units_4': 15,
 'units_5': 11,
 'units_6': 1,
 'units_7': 1,
 'units_8': 13,
 'units_9': 13,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0132'}

In [28]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

51/51 - 0s - loss: 0.1046 - accuracy: 0.9783 - 299ms/epoch - 6ms/step
Loss: 0.10458814352750778, Accuracy: 0.9782743453979492


In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_scaled[0])
hidden_nodes_layer1 =  21
hidden_nodes_layer2 = 17
hidden_nodes_layer3 = 23
hidden_nodes_layer4 = 23
hidden_nodes_layer5 = 15
hidden_nodes_layer6 = 11
hidden_nodes_layer7 = 1




output_dim = 1



nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="selu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="selu"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer7, activation="selu"))








# Output layer
nn_model.add(tf.keras.layers.Dense(output_dim, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 21)                105       
                                                                 
 dense_10 (Dense)            (None, 17)                374       
                                                                 
 dense_11 (Dense)            (None, 23)                414       
                                                                 
 dense_12 (Dense)            (None, 23)                552       
                                                                 
 dense_13 (Dense)            (None, 15)                360       
                                                                 
 dense_14 (Dense)            (None, 11)                176       
                                                                 
 dense_15 (Dense)            (None, 1)                

In [30]:
nn_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

In [31]:

# Train the model
fit_model = nn_model.fit(X_scaled, y_train, epochs=100, initial_epoch= 0)

Epoch 1/100
151/151 [==============================] - 2s 7ms/step - loss: 0.3939 - accuracy: 0.9570
Epoch 2/100
151/151 [==============================] - 1s 7ms/step - loss: 0.2973 - accuracy: 0.9768
Epoch 3/100
151/151 [==============================] - 1s 7ms/step - loss: 0.2523 - accuracy: 0.9756
Epoch 4/100
151/151 [==============================] - 1s 7ms/step - loss: 0.2142 - accuracy: 0.9756
Epoch 5/100
151/151 [==============================] - 1s 6ms/step - loss: 0.1842 - accuracy: 0.9760
Epoch 6/100
151/151 [==============================] - 1s 7ms/step - loss: 0.1616 - accuracy: 0.9766
Epoch 7/100
151/151 [==============================] - 1s 7ms/step - loss: 0.1453 - accuracy: 0.9768
Epoch 8/100
151/151 [==============================] - 1s 7ms/step - loss: 0.1340 - accuracy: 0.9766
Epoch 9/100
151/151 [==============================] - 1s 7ms/step - loss: 0.1260 - accuracy: 0.9764
Epoch 10/100
151/151 [==============================] - 1s 8ms/step - loss: 0.1207 - accura

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

51/51 - 0s - loss: 0.1014 - accuracy: 0.9770 - 328ms/epoch - 6ms/step
Loss: 0.10144199430942535, Accuracy: 0.9770328998565674


In [33]:
predictions = nn_model.predict(X_test_scaled)

51/51 [==============================] - 0s 2ms/step


In [34]:

# Assuming y_true contains the true labels and y_pred contains the predicted labels

#accuracy = accuracy_score(y_test, predictions)
#precision = precision_score(y_test, predictions)
#recall = recall_score(y_test, predictions)
#f1 = f1_score(y_test, predictions)
auc_roc = roc_auc_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)


In [35]:
print(f'AUC-ROC score: {auc_roc}')
print(f'MSE score: {mse}')
print(f'MAE score: {mae}')

AUC-ROC score: 0.985765459158351
MSE score: 0.02167165494497384
MAE score: 0.046413316670342736


In [25]:
nn_model.save('trained_model_president.h5')